IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M2\FAULT_M2_10


In [3]:
guasto = 2

VOLO m2 FAULT 10%

In [4]:
rcou_m2_fault10 = pd.read_csv("RCOU.csv")
rcou_m2_fault10 = rcou_m2_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m2_fault10 = rcou_m2_fault10[((rcou_m2_fault10['C9'] >= potenza) & (rcou_m2_fault10['C10'] >= potenza) & (rcou_m2_fault10['C11'] >= potenza) & (rcou_m2_fault10['C12'] >= potenza) & (rcou_m2_fault10['C13'] >= potenza) & (rcou_m2_fault10['C14']>= potenza))]
rcou_m2_fault10=rcou_m2_fault10.reset_index(drop=True)
display(rcou_m2_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,76591680,1355,1440,1435,1362,1393,1405
1,76688094,1360,1538,1465,1447,1371,1530
2,76708255,1418,1514,1474,1462,1423,1510
3,76739385,1527,1442,1476,1497,1516,1455
4,76749554,1507,1477,1499,1485,1513,1471
...,...,...,...,...,...,...,...
7760,182518531,1516,1424,1477,1466,1372,1557
7761,182575566,1449,1493,1536,1399,1368,1560
7762,182618295,1471,1457,1527,1392,1392,1527
7763,182682604,1496,1379,1460,1421,1365,1508


In [5]:
min=rcou_m2_fault10['TimeUS'][0]
max=rcou_m2_fault10['TimeUS'][len(rcou_m2_fault10)-1]
print(max)
print(min)

182693145
76591680


In [6]:
xkf1_m2_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m2_fault10 = xkf1_m2_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m2_fault10 = xkf1_m2_fault10[((xkf1_m2_fault10['TimeUS'] >= min) & (xkf1_m2_fault10['TimeUS'] <= max	))]
xkf1_m2_fault10 = xkf1_m2_fault10.reset_index(drop=True)
print(xkf1_m2_fault10)

         TimeUS  Roll  Pitch   Yaw
0      76600505 -0.11   3.26  3.62
1      76610567  0.25   3.25  3.58
2      76620889 -0.02   3.32  3.55
3      76630882 -0.35   3.39  3.58
4      76641090 -0.04   3.35  3.60
...         ...   ...    ...   ...
9638  182649204 -0.61  -0.99  0.82
9639  182659430 -0.54  -0.98  0.82
9640  182669677 -0.47  -0.99  0.83
9641  182680163 -0.43  -1.00  0.82
9642  182690890 -0.43  -1.05  0.82

[9643 rows x 4 columns]


In [7]:
att_m2_fault10 = pd.read_csv("ATT.csv")
att_m2_fault10 = att_m2_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m2_fault10 = att_m2_fault10[((att_m2_fault10['TimeUS'] >= min) & (att_m2_fault10['TimeUS'] <= max	))]
att_m2_fault10=att_m2_fault10.reset_index(drop=True)
print(att_m2_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0      76601803    -0.12 -0.11      0.44   3.26    3.67  3.62
1      76612061    -0.14  0.25      0.45   3.25    3.67  3.58
2      76622292    -0.17 -0.02      0.47   3.32    3.67  3.55
3      76632106    -0.20 -0.35      0.47   3.39    3.67  3.58
4      76643046    -0.22 -0.04      0.47   3.35    3.67  3.60
...         ...      ...   ...       ...    ...     ...   ...
9638  182650515     1.82 -0.61     -2.13  -0.99    1.57  0.82
9639  182660867     1.82 -0.54     -2.15  -0.98    1.57  0.82
9640  182671317     1.81 -0.47     -2.17  -0.99    1.57  0.83
9641  182681862     1.79 -0.43     -2.19  -1.00    1.57  0.82
9642  182692387     1.77 -0.43     -2.21  -1.05    1.56  0.82

[9643 rows x 7 columns]


In [8]:
from numpy import empty
to_add=[]

if len(att_m2_fault10)>len(xkf1_m2_fault10):
    to_add=att_m2_fault10[len(xkf1_m2_fault10):]
    att_m2_fault10=att_m2_fault10[:len(xkf1_m2_fault10)]

if len(xkf1_m2_fault10)>len(att_m2_fault10):
    to_add=xkf1_m2_fault10[len(att_m2_fault10):]

for idx,i in enumerate(att_m2_fault10['Roll']):
    if(xkf1_m2_fault10['Roll'][idx] != i):
        att_m2_fault10['Roll'][idx] = ((att_m2_fault10['Roll'][idx] + xkf1_m2_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m2_fault10['Pitch']):
    if(xkf1_m2_fault10['Pitch'][idx] != i):
        att_m2_fault10['Pitch'][idx] = ((att_m2_fault10['Pitch'][idx] + xkf1_m2_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m2_fault10['Yaw']):
    if(xkf1_m2_fault10['Yaw'][idx] != i):
        att_m2_fault10['Yaw'][idx] = ((att_m2_fault10['Yaw'][idx] + xkf1_m2_fault10['Yaw'][idx])/2)

if not empty(to_add):
    att_m2_fault10=pd.concat([att_m2_fault10,to_add])

print(att_m2_fault10)

C:\Users\chiar\AppData\Local\Temp\ipykernel_12936\173014739.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m2_fault10['Yaw'][idx] = ((att_m2_fault10['Yaw'][idx] + xkf1_m2_fault10['Yaw'][idx])/2)


         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0      76601803    -0.12 -0.11      0.44   3.26    3.67  3.62
1      76612061    -0.14  0.25      0.45   3.25    3.67  3.58
2      76622292    -0.17 -0.02      0.47   3.32    3.67  3.55
3      76632106    -0.20 -0.35      0.47   3.39    3.67  3.58
4      76643046    -0.22 -0.04      0.47   3.35    3.67  3.60
...         ...      ...   ...       ...    ...     ...   ...
9638  182650515     1.82 -0.61     -2.13  -0.99    1.57  0.82
9639  182660867     1.82 -0.54     -2.15  -0.98    1.57  0.82
9640  182671317     1.81 -0.47     -2.17  -0.99    1.57  0.83
9641  182681862     1.79 -0.43     -2.19  -1.00    1.57  0.82
9642  182692387     1.77 -0.43     -2.21  -1.05    1.56  0.82

[9643 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m2_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m2_fault10 = esc_0_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m2_fault10 = esc_0_m2_fault10[((esc_0_m2_fault10['TimeUS'] >= min) & (esc_0_m2_fault10['TimeUS'] <= max))]
esc_0_m2_fault10=esc_0_m2_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m2_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m2_fault10 = esc_1_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m2_fault10 = esc_1_m2_fault10[((esc_1_m2_fault10['TimeUS'] >= min) & (esc_1_m2_fault10['TimeUS'] <= max))]
esc_1_m2_fault10=esc_1_m2_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m2_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m2_fault10 = esc_2_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m2_fault10 = esc_2_m2_fault10[((esc_2_m2_fault10['TimeUS'] >= min) & (esc_2_m2_fault10['TimeUS'] <= max))]
esc_2_m2_fault10=esc_2_m2_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m2_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m2_fault10 = esc_3_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m2_fault10 = esc_3_m2_fault10[((esc_3_m2_fault10['TimeUS'] >= min) & (esc_3_m2_fault10['TimeUS'] <= max))]
esc_3_m2_fault10=esc_3_m2_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m2_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m2_fault10 = esc_4_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m2_fault10 = esc_4_m2_fault10[((esc_4_m2_fault10['TimeUS'] >= min) & (esc_4_m2_fault10['TimeUS'] <= max))]
esc_4_m2_fault10=esc_4_m2_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m2_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m2_fault10 = esc_5_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m2_fault10 = esc_5_m2_fault10[((esc_5_m2_fault10['TimeUS'] >= min) & (esc_5_m2_fault10['TimeUS'] <= max))]
esc_5_m2_fault10=esc_5_m2_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m2_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m2_fault10 = imu_0_m2_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m2_fault10 = imu_0_m2_fault10[((imu_0_m2_fault10['TimeUS'] >= min) & (imu_0_m2_fault10['TimeUS'] <= max))]

imu_1_m2_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m2_fault10 = imu_1_m2_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m2_fault10 = imu_1_m2_fault10[((imu_1_m2_fault10['TimeUS'] >= min) & (imu_1_m2_fault10['TimeUS'] <= max))]

imu_2_m2_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m2_fault10 = imu_2_m2_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m2_fault10 = imu_2_m2_fault10[((imu_2_m2_fault10['TimeUS'] >= min) & (imu_2_m2_fault10['TimeUS'] <= max))]

imu_m2_fault10 = pd.concat((imu_0_m2_fault10, imu_1_m2_fault10, imu_2_m2_fault10))
imu_m2_fault10=imu_m2_fault10.groupby(imu_m2_fault10.TimeUS, as_index=False).mean()

display(imu_m2_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,76596898,-0.087496,0.129375,0.002737,0.410752,1.279124,-9.849309
1,76599612,0.629721,-0.022170,-0.038268,0.697404,1.520396,-9.676227
2,76601568,0.734351,-0.198096,-0.042900,0.828583,0.903306,-9.486217
3,76604796,-0.051758,-0.113764,0.003014,0.711837,0.047788,-9.498156
4,76606627,-0.559301,0.035907,0.019815,0.574881,-0.004650,-9.492326
...,...,...,...,...,...,...,...
36636,182681634,0.085096,-0.008251,-0.013832,-0.205013,0.486183,-9.757404
36637,182684723,0.118494,-0.062658,0.006834,-0.134503,0.242348,-9.748803
36638,182686868,-0.020153,-0.089405,0.008694,-0.236891,-0.043895,-9.593975
36639,182689400,-0.083203,-0.084285,0.006058,-0.320326,-0.045759,-9.582758


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m2_fault10 = pd.merge_ordered(imu_m2_fault10,att_m2_fault10)
m2_fault10=m2_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_0_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_1_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_2_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_3_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_4_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_5_m2_fault10)
m2_fault10=m2_fault10.fillna(method="ffill").fillna(method="bfill")
m2_fault10 = pd.merge_ordered(m2_fault10, rcou_m2_fault10)
m2_fault10=m2_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m2_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m2_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m2_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m2_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m2_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m2_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m2_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m2_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

9643
10538
11433
12329
13212
14107
15008


In [13]:
m2_fault10 = m2_fault10[~m2_fault10.TimeUS.isin(index_to_remove)]
m2_fault10["TimeUS"] = m2_fault10["TimeUS"] - m2_fault10.iloc[0]["TimeUS"]
m2_fault10["TimeUS"] = m2_fault10["TimeUS"].astype(int)
m2_fault10["Guasto"] = guasto

In [14]:
from datetime import timedelta
m2_fault10=m2_fault10.reset_index(drop=True)
m2_fault10['TimeUS'] = pd.to_datetime(m2_fault10['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m2_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.087496,0.129375,0.002737,0.410752,1.279124,-9.849309,-0.12,-0.11,0.44,...,3.49,4061.0,3.81,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2
1,00:00:00.002714,0.629721,-0.022170,-0.038268,0.697404,1.520396,-9.676227,-0.12,-0.11,0.44,...,3.49,4061.0,3.81,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2
2,00:00:00.004670,0.734351,-0.198096,-0.042900,0.828583,0.903306,-9.486217,-0.12,-0.11,0.44,...,3.49,4061.0,3.81,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2
3,00:00:00.007898,-0.051758,-0.113764,0.003014,0.711837,0.047788,-9.498156,-0.12,-0.11,0.44,...,3.49,4061.0,3.81,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2
4,00:00:00.009729,-0.559301,0.035907,0.019815,0.574881,-0.004650,-9.492326,-0.12,-0.11,0.44,...,3.49,4061.0,3.81,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36636,00:01:46.084736,0.085096,-0.008251,-0.013832,-0.205013,0.486183,-9.757404,1.81,-0.47,-2.17,...,1.98,5398.0,2.79,1471.0,1457.0,1527.0,1392.0,1392.0,1527.0,2
36637,00:01:46.087825,0.118494,-0.062658,0.006834,-0.134503,0.242348,-9.748803,1.79,-0.43,-2.19,...,1.98,5398.0,2.79,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0,2
36638,00:01:46.089970,-0.020153,-0.089405,0.008694,-0.236891,-0.043895,-9.593975,1.79,-0.43,-2.19,...,1.98,5398.0,2.79,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0,2
36639,00:01:46.092502,-0.083203,-0.084285,0.006058,-0.320326,-0.045759,-9.582758,1.79,-0.43,-2.19,...,1.98,5398.0,2.79,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0,2


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m2_fault10)/350)):
        V1=m2_fault10[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_12936\1158890272.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_12936\1158890272.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,0.035226,0.051780,3.130169,0.229941,97.0,79.0,8.187140,7.125163,-0.058857,...,9124.370919,3.739174,0.241010,-0.916202,3.771173,155.0,154.0,57.856033,14.263781,2
1,1.0,0.034609,0.051766,3.118413,0.229819,93.0,76.0,8.146343,6.930852,-0.058333,...,9247.150848,3.740312,0.240780,-0.914814,3.772272,148.0,147.0,56.911758,13.531574,2
2,2.0,0.034035,0.051735,3.110183,0.229668,89.0,72.0,8.113586,7.189842,-0.057900,...,9355.188708,3.741445,0.240548,-0.913362,3.773364,151.0,150.0,55.937993,12.783909,2
3,3.0,0.033962,0.051591,3.126994,0.229343,95.0,79.0,8.251471,7.375108,-0.057600,...,9448.423541,3.742571,0.240315,-0.911849,3.774450,150.0,149.0,54.935571,12.022862,2
4,4.0,0.033994,0.051445,3.144086,0.229033,93.0,76.0,8.420312,7.262189,-0.057382,...,9526.685935,3.743690,0.240082,-0.910275,3.775529,151.0,150.0,53.905329,11.250457,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,99.0,-0.011650,0.059587,1.930746,0.244112,92.0,114.0,7.531282,6.973401,-0.042922,...,12489.442688,3.841889,0.226630,-0.665584,3.871206,-170.0,-172.0,56.228246,17.381858,2
100,100.0,-0.012224,0.059604,1.918672,0.244174,104.0,101.0,10.587730,8.288973,-0.042815,...,12541.192253,3.842971,0.226654,-0.663137,3.872283,-170.0,-172.0,57.061261,17.151476,2
101,101.0,-0.012045,0.059486,1.927503,0.243925,92.0,94.0,11.327731,10.021544,-0.043080,...,12579.964228,3.844049,0.226676,-0.660660,3.873356,-166.0,-168.0,57.872175,16.890009,2
102,102.0,-0.012298,0.059384,1.933135,0.243729,96.0,120.0,9.151912,7.696771,-0.043358,...,12605.688439,3.845121,0.226696,-0.658153,3.874423,-164.0,-166.0,58.660671,16.600958,2


In [18]:
path_file = path_file.replace(r"M2\FAULT_M2_10", "")
os.chdir(path_file)
df_merged.to_csv('m2_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
